# Import packages and database

In [170]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.segmentation as seg
from biogeme.expressions import Beta, Variable, Derive
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
from collections import namedtuple

data_file ='https://raw.githubusercontent.com/GustavePellier/MMOB/main/lpmc19.dat'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC

database = db.Database('LPMC', LPMC)
all_results = {}

# Model 0

We calculate the total public transport duration and the total driving cost

In [171]:
LPMC["dur_pt"]= LPMC["dur_pt_access"] + LPMC["dur_pt_rail"] + LPMC["dur_pt_bus"] + LPMC["dur_pt_int"] 
LPMC["cost_drive"] = LPMC["cost_driving_ccharge"] + LPMC["cost_driving_fuel"]

Some variables are created with the columns that seem to be useful

In [172]:
travel_mode=Variable('travel_mode')

dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')

pt_interchanges=Variable('pt_interchanges')

cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
cost_drive=Variable('cost_drive')
cost_pt=Variable('cost_transit')

time_walk=Variable('dur_walking')
time_cycle=Variable('dur_cycling')
time_pt=Variable('dur_pt')
time_drive=Variable('dur_driving')

female=Variable('female')
age=Variable('age')

There are 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [173]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

Utility functions creation

In [174]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

availability of each mode, all available here

In [175]:
av = {1: 1, 2: 1, 3: 1, 4:1}

The estimation results (parameter values, t-tests or p-values, null and final log likelihoods)

In [176]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.878619,0.107672,-36.022418,0.0
asc_drive,-1.295191,0.080505,-16.088354,0.0
asc_pt,-0.503354,0.054123,-9.300119,0.0
beta_cost,-0.193629,0.013958,-13.871889,0.0
beta_time,-5.495527,0.208596,-26.345342,0.0


In [177]:
res.compile_estimation_results(all_results)

(                                         Model0
 Number of estimated parameters                5
 Sample size                                5000
 Final log likelihood               -4587.818071
 Akaike Information Criterion        9185.636142
 Bayesian Information Criterion      9218.222108
 asc_cycle (t-test)                 -3.88  (-36)
 asc_drive (t-test)                -1.3  (-16.1)
 asc_pt (t-test)                  -0.503  (-9.3)
 beta_cost (t-test)              -0.194  (-13.9)
 beta_time (t-test)                -5.5  (-26.3),
 {'Model0': 'Model0'})

In [178]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1

We choose to do a specification for the time attribute, the cost stays generic in this model. Therefore we obtain a beta_time parameter for each transportation mode. 

In [179]:
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)

The utility functions are rewritten with the new parameters :

In [180]:
v_walk_model1= beta_time_walk * time_walk  
v_cycle_model1= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1= asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1= asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

The results are computed (parameter values, t-tests or p-values, null and final log likelihoods)

In [181]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model_1'
all_results['Model1'] = biogeme_model1.estimate()
results_specific_time = biogeme_model1.estimate()
results_specific_time.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.847224,0.202570,-23.928654,0.0
asc_drive,-1.969873,0.137929,-14.281808,0.0
asc_pt,-2.337196,0.139506,-16.753342,0.0
beta_cost,-0.182008,0.016087,-11.314047,0.0
beta_time_cycle,-5.312452,0.462002,-11.498762,0.0
beta_time_drive,-6.474307,0.378989,-17.083106,0.0
beta_time_pt,-3.538619,0.250656,-14.117426,0.0
beta_time_walk,-8.430765,0.418990,-20.121621,0.0


In [182]:
res.compile_estimation_results(all_results)

(                                         Model0           Model1
 Number of estimated parameters                5                8
 Sample size                                5000             5000
 Final log likelihood               -4587.818071     -4275.496542
 Akaike Information Criterion        9185.636142      8566.993084
 Bayesian Information Criterion      9218.222108       8619.13063
 asc_cycle (t-test)                 -3.88  (-36)   -4.85  (-23.9)
 asc_drive (t-test)                -1.3  (-16.1)   -1.97  (-14.3)
 asc_pt (t-test)                  -0.503  (-9.3)   -2.34  (-16.8)
 beta_cost (t-test)              -0.194  (-13.9)  -0.182  (-11.3)
 beta_time (t-test)                -5.5  (-26.3)                 
 beta_time_cycle (t-test)                          -5.31  (-11.5)
 beta_time_drive (t-test)                          -6.47  (-17.1)
 beta_time_pt (t-test)                             -3.54  (-14.1)
 beta_time_walk (t-test)                           -8.43  (-20.1),
 {'Model0

In [183]:
biogeme_model1.calculateNullLoglikelihood(av)

-6931.471805599917

We compare the 2 models with the likelihood ratio test because model 0 is a reduction of model 1.  

In [184]:
results_specific_time.likelihood_ratio_test(results_generic, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=624.6430583499878, threshold=7.814727903251179)

# Model 2

Using Model 1 as the base model, we choose to study how gender interact with the
ASCs and the alternative attributes. 

Lets start by segmenting the population by gender

In [185]:
gender_segmentation = seg.DiscreteSegmentationTuple(variable=female, mapping={1: 'female', 0: 'male'})

### ASC segmentation

In [186]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
segmented_asc_cycle = seg.Segmentation(asc_cycle,[gender_segmentation]).segmented_beta()

asc_pt = Beta('asc_pt', 0, None, None, 0)
segmented_asc_pt = seg.Segmentation(asc_pt,[gender_segmentation]).segmented_beta()

asc_drive = Beta('asc_drive', 0, None, None, 0)
segmented_asc_drive = seg.Segmentation(asc_drive,[gender_segmentation]).segmented_beta()

We redefine the value function of the 4 alternatives, introducing the segmented ASC and the segmented cost attribute

In [187]:
v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

We can compute the result of the new model

In [188]:
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model2'] = biogeme_model2.estimate()
results_segmented_gender = biogeme_model2.estimate()
results_segmented_gender.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-5.454910,0.253347,-21.531354,0.000000e+00
asc_cycle_male,1.051802,0.208766,5.038188,4.699608e-07
asc_drive,-1.893661,0.144300,-13.123043,0.000000e+00
asc_drive_male,-0.197962,0.095253,-2.078277,3.768381e-02
asc_pt,-2.208490,0.146370,-15.088352,0.000000e+00
asc_pt_male,-0.333165,0.099554,-3.346591,8.181180e-04
beta_cost,-0.182827,0.016154,-11.317822,0.000000e+00
beta_time_cycle,-5.580780,0.473118,-11.795745,0.000000e+00
beta_time_drive,-6.487957,0.380056,-17.071074,0.000000e+00
beta_time_pt,-3.527004,0.251404,-14.029204,0.000000e+00


In [189]:
biogeme_model2.calculateNullLoglikelihood(av)

-6931.471805599917

### Cost segmentation

And its interaction on the cost attribute in the car and public transport alternative.

In [190]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
segmented_beta_cost = seg.Segmentation(beta_cost, [gender_segmentation]).segmented_beta()

In [191]:
v_walk_model2_2= beta_time_walk * time_walk  
v_cycle_model2_2= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2_2= asc_pt + beta_time_pt * time_pt + segmented_beta_cost * cost_pt
v_drive_model2_2= asc_drive + beta_time_drive * time_drive + segmented_beta_cost * cost_drive

We can compute the result of the new model

In [192]:
V_model2_2 = {1: v_walk_model2_2 , 2: v_cycle_model2_2, 3: v_pt_model2_2, 4: v_drive_model2_2}
logprob_model2_2 = models.loglogit(V_model2_2, av, travel_mode)
biogeme_model2_2 = bio.BIOGEME(database, logprob_model2_2)
biogeme_model2_2.modelName = 'Model_2_2'
all_results['Model2_2'] = biogeme_model2_2.estimate()
results_segmented_gender_2 = biogeme_model2_2.estimate()
results_segmented_gender_2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.843766,0.202192,-23.956237,0.000000e+00
asc_drive,-1.971716,0.137731,-14.315753,0.000000e+00
asc_pt,-2.343113,0.139307,-16.819783,0.000000e+00
beta_cost,-0.151590,0.018749,-8.085464,6.661338e-16
beta_cost_male,-0.065393,0.031840,-2.053794,3.999566e-02
beta_time_cycle,-5.348638,0.459249,-11.646498,0.000000e+00
beta_time_drive,-6.483025,0.380413,-17.042049,0.000000e+00
beta_time_pt,-3.536891,0.250704,-14.107830,0.000000e+00
beta_time_walk,-8.435602,0.418400,-20.161583,0.000000e+00


In [193]:
biogeme_model2_2.calculateNullLoglikelihood(av)

-6931.471805599917

### Compiling all results from the different models

In [194]:
res.compile_estimation_results(all_results)

(                                         Model0           Model1  \
 Number of estimated parameters                5                8   
 Sample size                                5000             5000   
 Final log likelihood               -4587.818071     -4275.496542   
 Akaike Information Criterion        9185.636142      8566.993084   
 Bayesian Information Criterion      9218.222108       8619.13063   
 asc_cycle (t-test)                 -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                  -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)              -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                -5.5  (-26.3)                    
 beta_time_cycle (t-test)                          -5.31  (-11.5)   
 beta_time_drive (t-test)                          -6.47  (-17.1)   
 beta_time_pt (t-test)                             -3.54  (-14.1)   
 beta_time_walk (t-test)          

### Likelihood ratio test to check if model_2 and model_2_2 are equivalent or not

In [195]:
results_segmented_gender.likelihood_ratio_test(results_segmented_gender_2, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=55.92757888516644, threshold=5.991464547107979)

Model_2 is preferred because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2

### Comparison with model 1

In [197]:
results_segmented_gender.likelihood_ratio_test(results_specific_time, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=60.51262152961135, threshold=7.814727903251179)

Model_2 is preferred to Model_1 because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2.

# Model 3